Das brauche ich um's nicht zu vergessen.

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns


1. Originaldaten importieren

In [29]:
df = pd.read_csv('/Users/whypk/01Projekte/air_quality/data/waqi-covid19-airqualitydata-2025.csv', comment='#')
df.sample(50)
df.shape

(2321072, 9)

2. Spalten löschen
Wir konzentrieren uns bei der Analyse auf den Median der jeweiligen Messungen, daher kann auf die Spalten min und max verzichtet werden.
Die Berechnung der Varianz können wir nicht nachvollziehen und verzichten auch darauf.

In [30]:
df_small_24 = df.drop(columns=['variance', 'min', 'max'])
df_small_24.sample(20)

,Date,Country,City,Specie,count,median
1540002,2024-05-10,CN,Xuchang,temperature,72,25.0
1775810,2024-12-26,CY,Nicosia,wind-speed,40,2.0
453464,2025-02-06,FR,Montpellier,humidity,72,71.0
390230,2024-01-22,DE,Augsburg,co,14,0.1
2302261,2024-03-19,IL,Jerusalem,wind-gust,120,6.7
1129534,2025-01-04,IT,Rome,no2,192,13.8
2061098,2025-01-24,JP,Kyoto,co,344,5.6
31543,2024-01-05,TW,Keelung,o3,45,21.2
824193,2024-07-18,IN,Nagpur,co,10,3.8
172181,2024-07-30,US,Detroit,o3,23,22.4


In [31]:
#df_small_24.nunique('City')
df_small_24['City'].nunique()

579

3. Eine Stadt/ Land und zwar die mit den meisten Messwerten.

In [32]:
# Which cities per country have the highest number of measurements?
city_counts = df_small_24.groupby(["Country", "City"]).size().reset_index(name="count")
city_counts.sample(10)

,Country,City,count
185,FR,Limoges,3470
290,IR,Zanjān,3178
163,ES,Pamplona,4108
35,BR,Vitória,4852
397,NL,Eindhoven,3717
409,NP,Biratnagar,2578
311,JP,Kanazawa,5202
575,ZA,Klerksdorp,958
111,CN,Xuchang,4373
440,PT,Lisbon,3994


In [33]:
# Für jedes Land die Stadt mit den meisten Einträgen finden
max_city_per_country = city_counts.loc[city_counts.groupby("Country")["count"].idxmax()]
max_city_per_country.sample(10)


,Country,City,count
229,GT,Guatemala City,2575
19,BA,Sarajevo,4665
410,NP,Kathmandu,3119
288,IR,Tehran,4163
362,LK,Colombo,2324
469,SG,Singapore,4222
31,BG,Sofia,3706
419,PH,Zamboanga,900
366,MN,Ulan Bator,4563
2,AR,Buenos Aires,4103


In [34]:
# Ergebnisse anzeigen
print(max_city_per_country)
#als csv ablegen
max_city_per_country.to_csv('/Users/whypk/01Projekte/air_quality/data/max_city_per_country.csv', index=False)

    Country          City  count
0        AE     Abu Dhabi   4571
2        AR  Buenos Aires   4103
7        AT        Vienna   4849
9        AU      Brisbane   5599
19       BA      Sarajevo   4665
..      ...           ...    ...
519      US        Dallas   5143
564      UZ      Tashkent   2964
566      VN         Hanoi   4502
570      XK      Pristina   3555
578      ZA      Pretoria   4872

[90 rows x 3 columns]


4. Die ausgewählen Städte im df selektieren

In [35]:
# Aus max_city_per_country die Städte als Liste speichern
city_list_24=max_city_per_country['City'].tolist()

#aus df_small_24 nur die gewünschten Städte filtern
df_small_24_filtered = df_small_24[df_small_24['City'].isin(city_list_24)]
df_small_24_filtered

,Date,Country,City,Specie,count,median
0,2024-12-16,PE,Lima,o3,120,1.7
1,2025-01-04,PE,Lima,o3,192,2.6
2,2024-02-06,PE,Lima,o3,167,4.9
3,2024-04-19,PE,Lima,o3,144,5.4
4,2024-08-15,PE,Lima,o3,97,2.8
...,...,...,...,...,...,...
2321067,2024-03-19,IQ,Baghdad,wind-gust,2,13.8
2321068,2024-04-09,IQ,Baghdad,wind-gust,2,15.9
2321069,2024-06-25,IQ,Baghdad,wind-gust,2,9.7
2321070,2024-12-27,IQ,Baghdad,wind-gust,2,16.4


5. Counts ablegen, um im Nachhinein ggf. Fallzahlen kotrollieren zu können. 


@Mareike: müssen wir hier Zeilen mit zu wenig Fallzahlen schon löschen oder lieber später?

5. Spalte Species aufteilen

In [36]:
df_small_24_filtered = df_small_24_filtered.pivot(index=["Date", "Country", "City"], columns="Specie", values=["median", 'count']).reset_index()
df_small_24_filtered.columns = ['_'.join(col).strip('_') for col in df_small_24_filtered.columns]
df_small_24_filtered

,Date,Country,City,median_aqi,median_co,median_dew,median_humidity,median_mepaqi,median_no2,median_o3,...,count_pm10,count_pm25,count_precipitation,count_pressure,count_so2,count_temperature,count_uvi,count_wd,count_wind-gust,count_wind-speed
0,2024-01-01,AE,Abu Dhabi,NaN,0.1,19.0,73.0,NaN,8.1,21.2,...,120.0,24.0,NaN,192.0,120.0,192.0,NaN,NaN,NaN,184.0
1,2024-01-01,AR,Buenos Aires,NaN,1.9,15.8,63.0,NaN,4.6,12.2,...,75.0,24.0,2.0,92.0,8.0,92.0,NaN,21.0,48.0,92.0
2,2024-01-01,AT,Vienna,NaN,0.1,1.9,72.7,NaN,4.2,NaN,...,356.0,296.0,36.0,408.0,59.0,408.0,NaN,NaN,336.0,402.0
3,2024-01-01,AU,Brisbane,NaN,1.5,20.4,87.0,NaN,1.9,10.6,...,215.0,212.0,102.0,216.0,5.0,208.0,NaN,216.0,26.0,216.0
4,2024-01-01,BA,Sarajevo,NaN,1.6,6.0,96.5,NaN,15.6,25.2,...,121.0,98.0,6.0,144.0,115.0,144.0,NaN,NaN,6.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35712,2025-02-18,MN,Ulan Bator,NaN,13.0,-28.0,69.0,NaN,39.0,7.0,...,3.0,3.0,NaN,10.0,3.0,10.0,NaN,NaN,NaN,10.0
35713,2025-02-18,MY,Kuala Lumpur,65.0,NaN,25.0,94.0,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,2.0
35714,2025-02-18,NZ,Auckland,NaN,NaN,NaN,87.9,NaN,1.1,NaN,...,16.0,7.0,NaN,21.0,7.0,33.0,NaN,35.0,20.0,35.0
35715,2025-02-18,SG,Singapore,NaN,5.0,24.0,88.0,NaN,NaN,6.0,...,5.0,5.0,NaN,5.0,3.0,5.0,NaN,NaN,NaN,5.0


7. counts-Spalten löschen

In [38]:
df_small_24_filtered = df_small_24_filtered.drop(columns=[col for col in df_small_24_filtered.columns if col.startswith('count_')])
df_small_24_filtered.sample(20)

,Date,Country,City,aqi,co,dew,humidity,mepaqi,no2,o3,...,pm10,pm25,precipitation,pressure,so2,temperature,uvi,wd,wind-gust,wind-speed
27472,2024-11-15,HU,Miskolc,NaN,6.9,-0.3,87.1,NaN,7.8,4.7,...,32.0,89.0,NaN,998.1,2.6,1.4,NaN,NaN,3.4,1.0
16744,2024-07-15,PK,Islamabad,NaN,NaN,NaN,48.9,NaN,NaN,NaN,...,NaN,76.0,NaN,1003.6,NaN,33.4,NaN,NaN,4.4,1.3
8716,2024-04-14,DE,Berlin,NaN,0.1,4.5,62.2,NaN,3.9,30.0,...,12.0,25.0,NaN,1016.5,1.1,16.0,NaN,NaN,13.6,8.4
23248,2024-09-27,TJ,Dushanbe,NaN,NaN,12.5,64.0,NaN,NaN,NaN,...,NaN,162.0,NaN,1017.0,NaN,19.0,NaN,NaN,NaN,3.0
9141,2024-04-19,CL,Calama,NaN,0.9,-7.0,21.0,NaN,10.4,20.0,...,27.0,25.0,NaN,1026.0,1.8,13.0,NaN,NaN,NaN,8.7
29733,2024-12-11,IR,Tehran,NaN,37.0,-2.0,58.0,NaN,83.0,NaN,...,89.0,167.0,NaN,1028.0,32.0,5.0,NaN,NaN,NaN,1.5
403,2024-01-05,PR,San Juan,NaN,NaN,NaN,83.5,NaN,NaN,NaN,...,NaN,34.0,NaN,1008.4,NaN,8.6,NaN,NaN,4.3,0.6
28190,2024-11-23,RO,Bucharest,NaN,NaN,-3.3,64.0,NaN,5.1,11.6,...,9.0,32.0,99.1,1008.6,3.4,2.7,NaN,NaN,13.0,3.5
18947,2024-08-09,TM,Ashgabat,NaN,NaN,17.0,38.0,NaN,NaN,NaN,...,NaN,85.0,NaN,1009.0,NaN,33.0,NaN,NaN,NaN,3.0
28786,2024-11-30,MM,Yangon,NaN,NaN,24.0,83.0,NaN,NaN,NaN,...,NaN,78.0,NaN,1011.0,NaN,27.0,NaN,NaN,NaN,1.5


8. Median-Spalten umbenennen

In [39]:
df_small_24_filtered.rename(columns={col: col[len('median_'):] for col in df_small_24_filtered.columns if col.startswith('median')}, inplace=True)
df_small_24_filtered.sample(20)

,Date,Country,City,aqi,co,dew,humidity,mepaqi,no2,o3,...,pm10,pm25,precipitation,pressure,so2,temperature,uvi,wd,wind-gust,wind-speed
15381,2024-06-30,CA,Vancouver,NaN,0.1,14.0,80.7,NaN,2.2,5.7,...,7.0,18.0,NaN,1018.2,0.2,18.3,NaN,NaN,3.5,0.7
30776,2024-12-23,IL,Tel Aviv,NaN,7.0,9.5,74.5,NaN,22.1,10.4,...,42.0,65.0,NaN,1015.0,2.3,15.0,NaN,NaN,3.2,2.3
2098,2024-01-26,CH,Zürich,NaN,NaN,6.0,85.3,NaN,10.1,14.8,...,6.0,2.0,NaN,1030.0,0.3,8.6,NaN,NaN,2.1,1.0
12833,2024-05-31,NO,Oslo,NaN,0.1,NaN,53.6,NaN,7.1,NaN,...,15.0,34.0,NaN,992.3,NaN,21.0,NaN,NaN,2.0,0.8
24373,2024-10-10,KG,Bishkek,NaN,NaN,6.0,48.0,NaN,NaN,NaN,...,NaN,30.0,NaN,1012.0,NaN,16.0,NaN,NaN,NaN,2.0
33085,2025-01-18,VN,Hanoi,NaN,6.0,10.5,73.5,NaN,1.0,4.5,...,89.0,184.0,NaN,1019.0,5.0,16.0,NaN,NaN,NaN,2.8
17235,2024-07-21,GH,Accra,NaN,NaN,23.0,88.0,NaN,NaN,NaN,...,NaN,61.0,NaN,1014.0,NaN,25.0,NaN,NaN,NaN,2.5
4332,2024-02-22,GR,Athens,NaN,0.1,9.0,77.0,NaN,13.8,26.9,...,26.0,70.0,NaN,1020.2,2.6,12.7,NaN,NaN,3.2,1.7
35112,2025-02-11,BA,Sarajevo,NaN,8.5,0.0,86.0,NaN,26.1,7.8,...,91.0,185.0,99.0,1026.0,57.0,2.5,NaN,NaN,NaN,0.7
32465,2025-01-11,TW,Taipei,NaN,3.0,8.6,81.0,NaN,9.2,24.8,...,16.0,30.0,NaN,1028.9,1.2,12.2,1.0,NaN,14.0,2.2
